<a href="https://colab.research.google.com/github/paro2708/SER517_Group35_Capstone/blob/GazeRefineNet/EyeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To add

# EyeNet

EyeNet model to be used to tranfer learning to GazeRefineNet model

In [ ]:
import torchvision.models as models
import torch
from torch import nn
import numpy as np
import math

In [ ]:
class EyeNet(nn.Module):
  def __init__(self, use_rnn=False):
    super(EyeNet, self).__init__()
    self.resnet = models.resnet18(
        pretrained=False,        # Not using pre-trained weights
        num_classes=512,        # (IMP)Number of output classes - needs to be defined based on number of eye features needed
        norm_layer=nn.InstanceNorm2d,  # Normalization layer - layers need to be added
    )
    self.use_rnn = use_rnn

    # Optional recurrent component
    if use_rnn:
            self.rnn = nn.LSTM(input_size=512, hidden_size=512, num_layers=2, batch_first=True)
            self.fc_gaze = nn.Linear(512, 3)  # Output size for gaze direction
    else:
      self.fc_gaze = nn.Linear(512, 3)  # Output size for gaze direction
    self.fc_pupil = nn.Linear(512, 1)  # Output size for pupil size

  def forward(self, x):
    features = self.resnet(x)
    if self.use_rnn:
        features, _ = self.rnn(features.unsqueeze(0))  # Add an extra dimension for the sequence length
        features = features.squeeze(0)  # Remove the extra dimension added for the sequence length
    gaze_direction = self.fc_gaze(features)
    pupil_size = self.fc_pupil(features)
    return gaze_direction, pupil_size

   # Define custom loss functions
  def gaze_loss(g, g_hat):
    loss = torch.mean(torch.acos(torch.sum(g * g_hat, dim=1) / (torch.norm(g, dim=1) * torch.norm(g_hat, dim=1))) * (180 / torch.pi))
    return loss

  def pupil_loss(p, p_hat):
    loss = torch.mean(torch.abs(p - p_hat))
    return loss

#To test if the models can run image data for eye coordinates
eyenet = EyeNet(use_rnn=True)  # You can set use_rnn to True or False based on your requirement

# Generate sample input data (e.g., random tensor with appropriate dimensions)
# assuming batch size = 1, 3 channels (RGB), and image size 224x224
input_data = torch.randn(1, 3, 224, 224)

# Pass the input data through the EyeNet model to get predictions
gaze_direction, pupil_size = eyenet(input_data)

magnitude = torch.norm(gaze_direction, p=2)

# Print the predictions
print("Predicted Gaze Direction:", gaze_direction)
print("Predicted Pupil Size:", pupil_size)
print("Normalized Gaze Direction Magnitude(in radians):", magnitude.item())




Predicted Gaze Direction: tensor([[-0.0259,  0.0109,  0.0062]], grad_fn=<AddmmBackward0>)
Predicted Pupil Size: tensor([[0.0443]], grad_fn=<AddmmBackward0>)
Normalized Gaze Direction Magnitude(in radians): 0.028716016560792923
